In [90]:
import pandas as pd
import resource 

file = 'sales.csv'
total_lines = 500_000

pred = (lambda x: x['Sales Channel'] == 'Offline')

**Ideas**

Set max memory usage and have chunksize dynamically adjust


In [92]:
%%time
SAMPLE_SIZE = 10
MAX_MEM_MB = 1000
MAX_MEM_BYTES = 1024*1024*MAX_MEM_MB

df_head = pd.DataFrame()
df_head = pd.read_csv(file, nrows=SAMPLE_SIZE)

avg_line_bytes = df_head.memory_usage(deep=True).sum()/SAMPLE_SIZE
print(f'avg line bytes {avg_line_bytes}')

total_mem_estimate = total_lines * avg_line_bytes
print(f'MEM estimate {total_mem_estimate}')

df_read_size = 0
lines_read = 0

df_total = pd.DataFrame()

while lines_read < total_lines:
#     print(f'lefft {total_lines -lines_read}')
    lines_to_read = min(int((MAX_MEM_BYTES - df_read_size) / avg_line_bytes), total_lines-lines_read)
    print(f'lines to read {lines_to_read}')

    df_chunk = pd.read_csv(file, header=0, skiprows=range(1,lines_read), nrows=lines_to_read)
    
#     print(df_chunk.shape)
    indices = df_chunk[pred].index
#     print(len(indices))
    df_chunk.drop(indices, inplace=True)
    chunk_mem = df_chunk.memory_usage(deep=True).sum()
    
    df_total = pd.concat([df_total, df_chunk])
    
    
    lines_read += lines_to_read
    df_read_size += chunk_mem
    print(f'Total Current Mem Usage  {df_read_size }')
#     print(f'lines read {lines_read}')

print(df_total.shape)

avg line bytes 535.5
MEM estimate 267750000.0
lines to read 500000
Total Current Mem Usage  132214208
(249839, 14)
CPU times: user 5.52 s, sys: 197 ms, total: 5.71 s
Wall time: 5.85 s


In [93]:
%%time

df_total = pd.DataFrame()

for df in pd.read_csv(file, iterator=True, chunksize=57000):
    indices = df[pred].index
    df.drop(indices, inplace=True)
    df_total = pd.concat([df_total, df_chunk])


CPU times: user 2.95 s, sys: 934 ms, total: 3.88 s
Wall time: 3.94 s


In [91]:
%%time

df = pd.read_csv(file)

indices = df[pred].index
# chunk_mem = df.memory_usage(deep=True).sum()
# print(chunk_mem)
df.drop(indices, inplace=True)
print(df.shape)

(249839, 14)
CPU times: user 931 ms, sys: 134 ms, total: 1.07 s
Wall time: 1.12 s
